In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
import os
import joblib

In [ ]:
#Create a function to create models for different stocks and save them to .keras files

scalers={}
training_data_lens={}

def create_models(stock_name):
    df= pd.read_csv(f"stock_data/{stock_name}_last_5_years.csv")
    df['Date']=pd.to_datetime(df['Date'], errors='coerce')

    target_col='Close'

    dataset=df[target_col].values.reshape(-1,1)
    training_data_len=int(np.ceil(len(dataset) * 0.95))
    training_data_lens[stock_name]=training_data_len
    # print(f"Training data length = {training_data_len}")


    #Preprocessing data (scaling)
    scaler=StandardScaler()
    scaled_data=scaler.fit_transform(dataset)
    scalers[stock_name]=scaler

    #Create training data
    train_data=scaled_data[:training_data_len]

    X_train=[]
    y_train=[]

    #Creating 60 days sliding window for LSTM
    for i in range(60, len(train_data)):
        X_train.append(train_data[i-60:i, 0])   #60 previous days
        y_train.append(train_data[i, 0])        #61st day
    
    #Reshaping data for LSTM input(samples, time_steps, features)
    X_train, y_train=np.array(X_train), np.array(y_train)
    X_train=np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    #Build LSTM model
    model=keras.Sequential()

    #First layer
    model.add(keras.layers.LSTM(units=64,return_sequences=True,input_shape=(X_train.shape[1],1)))

    #Second layer
    model.add(keras.layers.LSTM(units=64,return_sequences=False))

    #Third layer
    model.add(keras.layers.Dense(units=25))

    #Fourth layer
    model.add(keras.layers.Dropout(0.5))

    #Final output layer
    model.add(keras.layers.Dense(units=1))

    # model.summary()

    model.compile(optimizer='adam',loss='mae', metrics=[keras.metrics.RootMeanSquaredError()])

    model.fit(X_train,y_train, epochs=20, batch_size=32)

    model.save(f"models/{stock_name}_model.keras")

    joblib.dump(scaler, f"models/scalers/{stock_name}_scaler.joblib")

    return model



In [ ]:
# Build models and predict for multiple stocks and visualize the test results

def train_models_for_stocks(stock_name):
    print(f"Creating model for {stock_name}...")
    model=create_models(stock_name)
    print(f"Model for {stock_name} created and saved.\n")

    #Testing the model
    df= pd.read_csv(f"stock_data/{stock_name}_last_5_years.csv")
    df['Date']=pd.to_datetime(df['Date'], errors='coerce')

    target_col='Close'
    dataset=df[target_col].values.reshape(-1,1)
    training_data_len=training_data_lens[stock_name]

    #get the scaler object
    scaler=scalers[stock_name]

    test_data=dataset[training_data_len - 60:]
    scaled_test_data=scaler.transform(test_data)

    #Prepare the test data
    X_test=[]
    y_test=dataset[training_data_len:]

    for i in range(60, len(scaled_test_data)):
        X_test.append(scaled_test_data[i-60:i, 0])


    X_test=np.array(X_test)
    X_test=np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    #Make a predictions
    predictions=model.predict(X_test)
    predictions=scaler.inverse_transform(predictions)

    #Plotting the data and the predictions (saved in outputs folder)
    train=df[:training_data_len]
    valid=df[training_data_len:]

    valid['Predictions']=predictions

    plt.figure(figsize=(16,8))
    plt.title(f"{stock_name} Stock Price Prediction")
    plt.xlabel('Date', fontsize=18)
    plt.ylabel('Close Price USD ($)', fontsize=18)
    plt.plot(train['Date'], train['Close'], label='Train Close Price')
    plt.plot(valid['Date'], valid['Close'], label='Actual Close Price')
    plt.plot(valid['Date'], valid['Predictions'], label='Predicted Close Price')
    plt.legend()
    plt.savefig(f"outputs/{stock_name}/stock_price_prediction.png")
    plt.close()

    print(f"Prediction plot for {stock_name} saved in outputs folder.\n")


    #Evaluate the model
    rmse=np.sqrt(np.mean(((predictions - y_test)**2))) 
    print(f"Root Mean Squared Error for {stock_name}: {rmse}")

    print("\n---------------------------------\n")
    return rmse
    

In [ ]:
stocks_list=["AAPL","MSFT","GOOGL","AMZN","TSLA"]
# stocks_list=["AAPL"]
RMSE_results={}
for stock in stocks_list:
    RMSE_results[stock]=train_models_for_stocks(stock)


Creating model for AAPL...


c:\Users\Anshul Guleria\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 0.3009 - root_mean_squared_error: 0.4168
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.1987 - root_mean_squared_error: 0.2635
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.1961 - root_mean_squared_error: 0.2597
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.1825 - root_mean_squared_error: 0.2452
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.1766 - root_mean_squared_error: 0.2390
Epoch 6/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.1807 - root_mean_squared_error: 0.2451
Epoch 7/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.1667 - root_mean_squared_error: 0.2243
Epoch 8/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.1749 - root_mean_squared_error: 0.2402
Epoch 9/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.1647 - root_mean_squared_error: 0.2279
Epoch 10/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.1529 - root_mean_squared_error: 0.2119

C:\Users\Anshul Guleria\AppData\Local\Temp\ipykernel_18884\4046769420.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions']=predictions


In [10]:
#Display RMSE results for all stocks
print("RMSE Results for all stocks:")
for stock, rmse in RMSE_results.items():
    print(f"{stock}: {rmse}")

RMSE Results for all stocks:
AAPL: 6.019528987716889
